In [1]:
from http.server import SimpleHTTPRequestHandler,HTTPServer
from urllib import request,error
import urllib.parse
import http.cookiejar
import sys
import requests
from pycaret.classification import *
import pandas as pd
import logging
import re
from datetime import datetime



In [2]:
#Logging
logging.basicConfig(filename='server.log', level=logging.INFO, format='%(asctime)s - %(message)s')  


model = load_model('PclBuild')

cookies = http.cookiejar.CookieJar()



bad_words = ['sleep', 'drop', 'uid', 'select', 'waitfor', 'delay', 'system', 'union', 'order by', 'delete', 'group by', 'insert', 'or']
xss = ['script','img','a','javascript','svg','onclick']
cmd = ['id','ls','dir','ping','uname','exec','nc','bash']

Transformation Pipeline and Model Successfully Loaded


In [3]:
def cmdi_check(path,body):

    cmdi_patterns = [
      r'(?:;|&|\||`|\$\(.*\)|\b(?:exec|system|shell_exec|passthru|popen|proc_open|pcntl_exec|eval|etc|bin|ls|id|uname|dir|ping|nc|bash)\b)',
      r'(?:;|&|\||`|\$\(.*\)|\b(?:exec|system|shell_exec|passthru|popen|proc_open|pcntl_exec|eval|etc|bin|ls|id|uname|dir|ping|nc|bash)\b)\s*',
      r'(?:;|&|\||`|\$\(.*\)|\b(?:exec|system|shell_exec|passthru|popen|proc_open|pcntl_exec|eval|etc|bin|ls|id|uname|dir|ping|nc|bash)\b)\s*[\\"\']?[^\\n\\r]*',
      r'(?:;|&|\||`|\$\(.*\)|\b(?:exec|system|shell_exec|passthru|popen|proc_open|pcntl_exec|eval|etc|bin|ls|id|uname|dir|ping|nc|bash)\b)\s*[\\"\']?[^\\"\']*[\\"\']?',
      r'(?:;|&|\||`|\$\(.*\)|\b(?:exec|system|shell_exec|passthru|popen|proc_open|pcntl_exec|eval|etc|bin|ls|id|uname|dir|ping|nc|bash)\b)[\s\w]*',
      r'(?:;|&|\||`|\$\(.*\)|\b(?:exec|system|shell_exec|passthru|popen|proc_open|pcntl_exec|eval|etc|bin|ls|id|uname|dir|ping|nc|bash)\b)[^\r\n]*'

    ]

    counter = 0
    #cmdi_pattern = r'(?:;|&|\||`|\$\(|\b(?:exec|system|shell_exec|passthru|popen|proc_open|pcntl_exec|eval|etc|bin|ls|id|uname|dir|ping|nc|bash)\b)'
    for pattern in cmdi_patterns:

        if re.search(pattern,path) or  re.search(pattern,body):
            counter+=1
    return counter


def regex_xss_check(path):
     
    xss_patterns = [
        r'<script\s[^>]*>[\s\S]*?<',
        r'(on\w+)=["\'](.*?)["\']',
        r'<img\s[^>]*\son\w+=["\'](.*?)["\'][^>]*>',
        r'href=["\'](javascript:[^"\']+)["\']',
        r'\bjavascript:\s*[^;]+;'
    ]
    count = 0
     

    for pattern in xss_patterns:
        if re.search(pattern, path):
            count=count+1
            

    return count





def extractDF(method,path,body):
    path = urllib.parse.unquote(path)
    single_quotes = path.count("'") + body.count("'")
    double_quotes = path.count('"') + body.count('"')
    dashes = path.count('--') + body.count('--')
    braces = path.count('{') + path.count('}') + path.count('(') + body.count('{') + body.count('}') + body.count('(')
    spaces = path.count(' ') + body.count(" ")
    tags = path.count('<') + path.count('>')+ body.count('<')+body.count('>')
    colons = path.count(';')+body.count(';')
    backtick = path.count('`')+body.count('`')
    bad_words_count1 = sum(1 for word in bad_words if re.search(r'\b' + re.escape(word) + r'\b', path, re.IGNORECASE))
    bad_words_count2 = sum(1 for word in bad_words if re.search(r'\b' + re.escape(word) + r'\b', body, re.IGNORECASE))
    xss_count1 = sum(1 for word in xss if re.search(r'\b' + re.escape(word) + r'\b', path, re.IGNORECASE))
    xss_count2 = sum(1 for word in xss if re.search(r'\b' + re.escape(word) + r'\b', body, re.IGNORECASE))
    bad_words_count = bad_words_count1+ bad_words_count2
    xss_count = xss_count1+xss_count2
    xss_check = regex_xss_check(path)
    cmdi_c = cmdi_check(path,body)
    cmdi_words1 = sum(1 for word in cmd if re.search(r'\b' + re.escape(word) + r'\b', path, re.IGNORECASE))
    cmdi_words2 = sum(1 for word in cmd if re.search(r'\b' + re.escape(word) + r'\b', body, re.IGNORECASE))
    cmdi_words = cmdi_words1+cmdi_words2



    datas = [single_quotes, double_quotes, dashes, braces, spaces,tags,colons,backtick, bad_words_count,xss_check,xss_count,cmdi_words,cmdi_c]
    
    log_data = [method,path,body]
    
    input = pd.DataFrame([datas], columns=['Single Quotes', 'Double Quotes', 'Dashes', 'Braces', 'Spaces','Tags','Colons','Backtick', 'Bad Words','XSS Check','XSS Word','Cmdi Word','Cmdi Check'])
    
    log = pd.DataFrame([log_data],columns=['Method','Path','Body'])
    return input,log

def prediction(input):
    prediction = predict_model(model,data=input)
 
    return prediction


In [4]:
import logging
from http.server import SimpleHTTPRequestHandler, HTTPServer
from datetime import datetime
import pandas as pd
from urllib import request
import requests
import socket

# Configure logging
logging.basicConfig(filename='http_traffic.log', level=logging.INFO, format='%(asctime)s - %(message)s')

# Define your functions here

class SimpleHttpProxy(SimpleHTTPRequestHandler):

    @classmethod
    def set_routes(cls, proxy_routes):
        cls.proxy_routes = proxy_routes

    def log_request_info(self, method, path, prediction, ip, status_code):
        log_message = f"Method: {method}, Path: {path}, Prediction: {prediction}, IP: {ip}, Status Code: {status_code}\n"
        with open('http_traffic.log', 'a') as log_file:
            log_file.write(log_message)
        data = {
        'method': method,
        'path': path,
        'prediction': prediction,
        'ip': ip,
        'statusCode': status_code
        }
        # Replace 'https://api.example.com' with your API endpoint
        api_endpoint = 'http://localhost:5000/api/logMessage'
        try:
            response = requests.post(api_endpoint, json=data)
            if response.status_code == 200:
                print("Log data sent to API successfully.")
            else:
                print(f"Failed to send log data to API. Status code: {response.status_code}")
        except Exception as e:
            print(f"Error occurred while sending log data to API: {e}")



    def do_GET(self) -> None:
        try:
            body = ''
            path = self.path
            method = self.command
            headers = self.headers
            full_url = f'{path}'
            
            response = request.urlopen(full_url)
            status_code = response.status
            content = response.read()
    
            inp, log = extractDF(method, path, body)
            pred = prediction(inp)
    
            result = pd.concat([log, pred], axis=1)
            result.to_csv('log_dataset.csv', mode='a', header=False, index=False)
    
            pp = pred['prediction_label'].to_string()
            
            ip = self.client_address[0]
            self.log_request_info(method, path, pp, ip, status_code)
    
            if 'good' in pp:
                self.send_response(200)
                self.send_header('Access-Control-Allow-Origin', '*')  # Allow requests from any origin
                self.end_headers()
                self.wfile.write(content)
            else:
                self.send_response(403)
                self.end_headers()
                # Custom HTML response
                custom_html = f"""
                <!DOCTYPE html>
                <html lang="en" style="height: 95%;">
                <head>
                    <meta charset="utf-8">
                    <meta name="viewport" content="width=device-width, initial-scale=1.0">
                    <title>403 Forbidden</title>
                    <style>
                        p {{
                            line-height: 20px;
                        }};
                    </style>
                </head>
                <body style="font-family: sans-serif; height: 100%;">
                    <div style="display: flex; justify-content: center; align-items: center; height: 100%;">
                        <div style="max-width: 90%; word-wrap: break-word;">
                            <h1 style="margin-bottom: 10px;">🛑 Request blocked</h1>
                            <h3 style="font-weight: normal; margin-top: 0px; margin-bottom: 5px; margin-left: 52px;">403 Forbidden</h3>
                            <hr style="margin-top: 1rem; margin-bottom: 1rem; border: 0; border-top: 1px solid rgba(0, 0, 0, 0.1);">
                            <p>This website uses a firewall to protect itself from online attacks.<br>
                                You have sent a suspicious request, therefore your request has been blocked.<br>
                                <strong>This is a custom blocked page.</strong></p>
                            <hr style="margin-top: 1rem; margin-bottom: 1rem; border: 0; border-top: 1px solid rgba(0, 0, 0, 0.1);">
                            <p id="dynamicContent">Time: <span id="dateTime"></span><br>
                Your IP: <span id="ip"></span><br>
                Country: <span id="country"></span><br>
                Region: <span id="regionName"></span><br>
                City: <span id="city"></span><br>
                Zip: <span id="zip"></span><br>
                Latitude: <span id="lat"></span><br>
                Longitude: <span id="lon"></span><br>                
                ISP: <span id="isp"></span><br>
                Organization: <span id="org"></span><br>
                                
                                Reference ID: <span id="referenceID"></span><br>
                                Attack Type: {pp.split()[-1].upper()}</p>
                        </div>
                    </div>
                    <script>
                    const referenceID = generateUID();
                        // Function to generate random UID
                        function generateUID() {{
                            return 'xxxxxxxx-xxxx-4xxx-yxxx-xxxxxxxxxxxx'.replace(/[xy]/g, function(c) {{
                                var r = Math.random() * 16 | 0,
                                    v = c == 'x' ? r : (r & 0x3 | 0x8);
                                return v.toString(16);
                            }});
                        }}
                    
                        // Function to get IP address
                        async function getIP(attackType) {{
                            const response = await fetch('http://api.ipify.org?format=json');
                            const data = await response.json();
                            const ip_data = await fetch(`http://ip-api.com/json/${{data.ip}}`);
                            const ip_data_response = await ip_data.json();
                            const {{query,country,regionName,city,zip,lat,lon,isp,org}} = ip_data_response;
                            const responseObject = {{referenceID,query,country,regionName,city,zip,lat,lon,isp,org, attackType}};
            
                            try {{
                                const response = await fetch('http://localhost:5000/api/logData', {{
                                    method: 'POST',
                                    headers: {{
                                        'Content-Type': 'application/json'
                                    }},
                                    body: JSON.stringify(responseObject)
                                }});
                                console.log('Data sent successfully:', await response.text());
                            }} catch (error) {{
                                console.error('Error sending data:', error);
                            }}
                            
                            return responseObject;
                        }}
                    
                        document.getElementById('dateTime').innerText = new Date().toLocaleString();
                        getIP('{pp.split()[-1].upper()}').then(ip => {{
                            document.getElementById('ip').innerText = ip.query;
                            document.getElementById('country').innerText = ip.country;
                            document.getElementById('regionName').innerText = ip.regionName;
                            document.getElementById('city').innerText = ip.city;
                            document.getElementById('zip').innerText = ip.zip;
                            document.getElementById('lat').innerText = ip.lat;
                            document.getElementById('lon').innerText = ip.lon;
                            document.getElementById('isp').innerText = ip.isp;
                            document.getElementById('org').innerText = ip.org;
                        }});
                        document.getElementById('referenceID').innerText = referenceID;
                    </script>
                </body>
                </html>
                """
                self.wfile.write(custom_html.encode('utf-8'))
    
                try:
                    resp = requests.post(nttfy_docker, data=data)
                    resp.raise_for_status()
                except requests.exceptions.HTTPError as err:
                    print(f"HTTP Error {resp.status_code}: {err}")
                except requests.exceptions.RequestException as err:
                    print(f"Request Error: {err}")
        except Exception as e:
            logging.error(f"Error in the GET request: {str(e)}")

    def do_POST(self) -> None:
        try:
            content_length = int(self.headers['Content-Length'])
            post_body = self.rfile.read(content_length).decode('utf-8')
            path = self.path
            method = self.command
            headers = self.headers
            full_url = f'{path}'
            body = post_body
            
            response = request.urlopen(full_url)
            status_code = response.status
            content = response.read()

            inp,log = extractDF(method,path,body)
            
            pred = prediction(inp)

            result = pd.concat([log,pred],axis=1)

            result.to_csv('log_dataset.csv',mode='a',header=False,index=False) 
        
            pp = pred['prediction_label'].to_string()
            
            ip = self.client_address[0]
            self.log_request_info(method, path, pp, ip, status_code)
            
            if 'good' in pp:
                self.send_response(200)
                self.send_header('Access-Control-Allow-Origin', '*')  # Allow requests from any origin
                self.end_headers()
                self.wfile.write(content)
                
            else:
                
                self.send_response(403)
                self.end_headers()
                self.wfile.write(b"Blocked by THE AI Firewall - contact @akashkampili")
                
                try:
                    resp = requests.post(nttfy_docker,data=data)
                    resp.raise_for_status()
                except requests.exceptions.HTTPError as err:
                    print(f"HTTP Error {resp.status_code}: {err}")
    		        
                except requests.exceptions.RequestException as err:
                    print(f"Request Error: {err}")
        except Exception as e:
            logging.error(f"Error in POST requests: {str(e)}")


In [ ]:
if __name__ == '__main__':
    server_address = ('', 1234)
    httpd = HTTPServer(server_address, SimpleHttpProxy)
    httpd.serve_forever()
    print("Server Started on port: 1234")

Log data sent to API successfully.


127.0.0.1 - - [08/May/2024 10:24:41] "GET http://detectportal.firefox.com/canonical.html HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2024 10:24:41] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [08/May/2024 10:24:41] "CONNECT incoming.telemetry.mozilla.org:443 HTTP/1.1" 501 -


Log data sent to API successfully.


127.0.0.1 - - [08/May/2024 10:24:47] "GET http://detectportal.firefox.com/success.txt?ipv4 HTTP/1.1" 200 -


Log data sent to API successfully.


...

127.0.0.1 - - [08/May/2024 11:47:42] "GET http://demo.testfire.net/search.jsp?query=-1%20UNION%20SELECT%201%20INTO%20@,@ HTTP/1.1" 403 -


Log data sent to API successfully.


127.0.0.1 - - [08/May/2024 11:47:43] "GET http://api.ipify.org/?format=json HTTP/1.1" 200 -


Log data sent to API successfully.


127.0.0.1 - - [08/May/2024 11:47:44] "GET http://ip-api.com/json/122.187.97.126 HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2024 11:48:24] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [08/May/2024 11:48:24] "CONNECT shavar.services.mozilla.com:443 HTTP/1.1" 501 -
127.0.0.1 - - [08/May/2024 11:50:39] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [08/May/2024 11:50:39] "CONNECT safebrowsing.googleapis.com:443 HTTP/1.1" 501 -
127.0.0.1 - - [08/May/2024 11:51:47] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [08/May/2024 11:51:47] "CONNECT spocs.getpocket.com:443 HTTP/1.1" 501 -
127.0.0.1 - - [08/May/2024 11:51:47] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [08/May/2024 11:51:47] "CONNECT firefox-api-proxy.cdn.mozilla.net:443 HTTP/1.1" 501 -
127.0.0.1 - - [08/May/2024 11:56:47] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [08/May/2024 11:56:47] "CONNECT contile.services.mozilla.com:443 HTTP/1.1" 501 -
127.0